#### 1. Load & Explore Raw Data

In [1]:
import pandas as pd
from pathlib import Path

DATA_DIR = Path("../data")
RAW_FILE = DATA_DIR / "ethio_interior_raw_2merkato_20260219_1644.csv"

df = pd.read_csv(RAW_FILE, parse_dates=["scraped_at"])
print(f"Loaded {len(df)} rows")
display(df.head(10))

Loaded 299 rows


,material_name,price_etb,price_text,unit,last_checked,detail_url,category,source,scraped_at
0,Clear Glass - 3mm thick,1202.0,"1,202.00 Brper m2",m2,"May 29, 2024",NaN,Finishing,2merkato.com,2026-02-19 16:44:00
1,Clear Glass - 4mm thick,1537.0,"1,537.00 Brper m2",m2,"May 29, 2024",NaN,Finishing,2merkato.com,2026-02-19 16:44:00
2,Clear Glass - 5mm thick,1767.0,"1,767.00 Brper m2",m2,"May 29, 2024",NaN,Finishing,2merkato.com,2026-02-19 16:44:00
3,Clear Glass - 6mm thick,2495.0,"2,495.00 Brper m2",m2,"May 29, 2024",NaN,Finishing,2merkato.com,2026-02-19 16:44:00
4,Figured Glass - 4mm thick,1776.0,"1,776.00 Brper m2",m2,"May 29, 2024",NaN,Finishing,2merkato.com,2026-02-19 16:44:00
5,Clear and Colored Glass: 5mm,1900.0,"1,900.00 Brper m2",m2,"May 29, 2024",NaN,Finishing,2merkato.com,2026-02-19 16:44:00
6,Clear and Colored Glass: 6mm,1350.0,"1,350.00 Brper m2",m2,"Dec 1, 2023",NaN,Finishing,2merkato.com,2026-02-19 16:44:00
7,Figured Glass - 5mm thick,1919.0,"1,919.00 Brper m2",m2,"May 29, 2024",NaN,Finishing,2merkato.com,2026-02-19 16:44:00
8,Frosted Glass - 3mm thick,1967.0,"1,967.00 Brper m2",m2,"May 29, 2024",NaN,Finishing,2merkato.com,2026-02-19 16:44:00
9,Frosted Glass - 4mm thick,1120.0,"1,120.00 Brper m2",m2,"May 29, 2024",NaN,Finishing,2merkato.com,2026-02-19 16:44:00


In [2]:
print("\nMissing values:")
print(df.isna().sum())


Missing values:
material_name      0
price_etb          0
price_text         0
unit               0
last_checked       0
detail_url       299
category           0
source             0
scraped_at         0
dtype: int64


In [3]:
print("\nPrice stats:")
print(df["price_etb"].describe())


Price stats:
count       299.000000
mean       3559.063880
std       14449.810958
min          15.000000
25%         350.000000
50%         815.000000
75%        1794.000000
max      146500.000000
Name: price_etb, dtype: float64


#### Cleaning & Feature Extraction



In [9]:
df_clean = df.copy()

print("Before cleaning:")
print(f"Rows: {len(df_clean)}")
print("Missing price_etb:", df_clean["price_etb"].isna().sum())
print("Missing material_name:", df_clean["material_name"].isna().sum())

# 1. Handle missing prices
median_per_cat = df_clean.groupby("category")["price_etb"].median()
df_clean["price_etb"] = df_clean.apply(
    lambda row: median_per_cat[row["category"]] if pd.isna(row["price_etb"]) else row["price_etb"],
    axis=1
)

# Drop rows that still have no price
df_clean = df_clean.dropna(subset=["price_etb"]).copy()

# 2. Normalize units
unit_map = {
    "m2": "square meter",
    "m²": "square meter",
    "m": "meter",
    "pcs": "piece",
    "pc": "piece",
    "bag": "bag",
    "kg": "kilogram",
    "ltr": "liter",
    "liter": "liter",
    "qt": "quintal",
    "quintal": "quintal",
    "no": "unit",
    "nos": "unit",
    "set": "set",
    "": "unknown",
    None: "unknown"
}

df_clean["unit_norm"] = df_clean["unit"].fillna("").str.lower().map(unit_map).fillna("unknown")

# 3. Improved keyword extraction from material_name
def extract_keywords(name):
    if not isinstance(name, str):
        return []
    
    name_lower = name.lower()
    keywords = set() 
    
    # Thickness / size
    if any(x in name_lower for x in ["thick", "mm", "cm", "inch", "size"]):
        keywords.add("size/thickness")
    
    # Color
    colors = ["white", "black", "blue", "red", "green", "grey", "gray", "brown", "yellow", "clear", "colored"]
    for color in colors:
        if color in name_lower:
            keywords.add(f"color:{color}")
    
    # Material type / finish
    if "frosted" in name_lower:
        keywords.add("frosted")
    if "clear" in name_lower:
        keywords.add("clear")
    if "textured" in name_lower or "texture" in name_lower:
        keywords.add("textured")
    if "matte" in name_lower:
        keywords.add("matte")
    if "glossy" in name_lower:
        keywords.add("glossy")
    if "ceramic" in name_lower or "porcelain" in name_lower:
        keywords.add("ceramic/porcelain")
    if "marble" in name_lower or "granite" in name_lower:
        keywords.add("stone/natural")
    
    # Brand / type hints
    if "opc" in name_lower or "ppc" in name_lower:
        keywords.add("cement type")
    if "galvanized" in name_lower:
        keywords.add("galvanized")
    
    return list(keywords)

df_clean["keywords"] = df_clean["material_name"].apply(extract_keywords)

# 4. Create rich text for embeddings (more context = better similarity)
df_clean["text_for_embedding"] = (
    df_clean["material_name"].fillna("unknown") + " " +
    df_clean["category"].fillna("") + " " +
    df_clean["unit_norm"].fillna("") + " " +
    df_clean["price_text"].fillna("") + " " +
    df_clean["keywords"].apply(lambda k: " ".join(k))  # add extracted keywords
).str.strip()

# Remove any rows with very short text
df_clean = df_clean[df_clean["text_for_embedding"].str.len() > 10].copy()

# Quick preview & stats
print("\nAfter cleaning:")
print(f"Rows: {len(df_clean)}")
print("Missing price_etb:", df_clean["price_etb"].isna().sum())
print("\nSample data (top 10):")
display(df_clean[["material_name", "category", "price_etb", "unit_norm", "keywords", "text_for_embedding"]].head(10))

print("\nMost common units after normalization:")
print(df_clean["unit_norm"].value_counts().head(10))

print("\nMost common keywords extracted:")
from collections import Counter
all_keywords = [kw for sublist in df_clean["keywords"] for kw in sublist]
print(Counter(all_keywords).most_common(10))

Before cleaning:
Rows: 299
Missing price_etb: 0
Missing material_name: 0

After cleaning:
Rows: 299
Missing price_etb: 0

Sample data (top 10):


,material_name,category,price_etb,unit_norm,keywords,text_for_embedding
0,Clear Glass - 3mm thick,Finishing,1202.0,square meter,"[clear, color:clear, size/thickness]",Clear Glass - 3mm thick Finishing square meter...
1,Clear Glass - 4mm thick,Finishing,1537.0,square meter,"[clear, color:clear, size/thickness]",Clear Glass - 4mm thick Finishing square meter...
2,Clear Glass - 5mm thick,Finishing,1767.0,square meter,"[clear, color:clear, size/thickness]",Clear Glass - 5mm thick Finishing square meter...
3,Clear Glass - 6mm thick,Finishing,2495.0,square meter,"[clear, color:clear, size/thickness]",Clear Glass - 6mm thick Finishing square meter...
4,Figured Glass - 4mm thick,Finishing,1776.0,square meter,"[color:red, size/thickness]",Figured Glass - 4mm thick Finishing square met...
5,Clear and Colored Glass: 5mm,Finishing,1900.0,square meter,"[color:clear, size/thickness, color:red, color...",Clear and Colored Glass: 5mm Finishing square...
6,Clear and Colored Glass: 6mm,Finishing,1350.0,square meter,"[color:clear, size/thickness, color:red, color...",Clear and Colored Glass: 6mm Finishing square ...
7,Figured Glass - 5mm thick,Finishing,1919.0,square meter,"[color:red, size/thickness]",Figured Glass - 5mm thick Finishing square met...
8,Frosted Glass - 3mm thick,Finishing,1967.0,square meter,"[frosted, size/thickness]",Frosted Glass - 3mm thick Finishing square met...
9,Frosted Glass - 4mm thick,Finishing,1120.0,square meter,"[frosted, size/thickness]",Frosted Glass - 4mm thick Finishing square met...



Most common units after normalization:
unit_norm
piece           155
unknown          62
kilogram         36
square meter     26
meter            20
Name: count, dtype: int64

Most common keywords extracted:
[('size/thickness', 160), ('galvanized', 48), ('clear', 9), ('color:clear', 9), ('color:red', 8), ('frosted', 4), ('cement type', 4), ('color:colored', 2), ('textured', 2), ('color:white', 2)]


#### Generate Embeddings (using Hugging Face)



In [11]:
from sentence_transformers import SentenceTransformer
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
import warnings

# Suppress Windows symlink warning (harmless)
warnings.filterwarnings("ignore", category=UserWarning, module="huggingface_hub.file_download")

# Load model
print("Loading sentence-transformers model...")
model = SentenceTransformer("all-MiniLM-L6-v2")
print("Model loaded successfully.")

# Generate embeddings
print(f"Generating embeddings for {len(df_clean)} materials...")

embeddings = model.encode(
    df_clean["text_for_embedding"].tolist(),
    show_progress_bar=True,
    batch_size=32,
    normalize_embeddings=True, 
    convert_to_numpy=True
)

# Add embeddings as list of arrays
df_clean["embedding"] = list(embeddings)

print(f"Success! Embeddings shape: {embeddings.shape} (rows × 384 dimensions)")

# Quick validation: similarity example 
if len(df_clean) >= 5:
    sim_matrix = cosine_similarity(embeddings[:5])
    print("\nSample cosine similarity matrix (first 5 items):")
    print(np.round(sim_matrix, 3))
    
    # Most similar to first item
    first_item = df_clean.iloc[0]["material_name"]
    sim_scores = sim_matrix[0]
    top_idx = np.argsort(sim_scores)[::-1][1:4]
    print(f"\nMost similar to '{first_item}':")
    for idx in top_idx:
        print(f"  → {df_clean.iloc[idx]['material_name']} (score: {sim_scores[idx]:.3f})")

# Save enriched data
timestamp = datetime.now().strftime("%Y%m%d_%H%M")
ENRICHED_CSV = DATA_DIR / f"interior_materials_enriched_{timestamp}.csv"
ENRICHED_PARQUET = DATA_DIR / f"interior_materials_enriched_{timestamp}.parquet"

df_clean.to_csv(ENRICHED_CSV, index=False, encoding="utf-8-sig")
df_clean.to_parquet(ENRICHED_PARQUET, index=False)

print(f"\nSaved enriched data:")
print(f"  CSV: {ENRICHED_CSV}")
print(f"  Parquet: {ENRICHED_PARQUET} (recommended for the next Phase)")

Loading sentence-transformers model...


Loading weights:   0%|          | 0/103 [00:00<?, ?it/s]

BertModel LOAD REPORT from: sentence-transformers/all-MiniLM-L6-v2
Key                     | Status     |  | 
------------------------+------------+--+-
embeddings.position_ids | UNEXPECTED |  | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.


Model loaded successfully.
Generating embeddings for 299 materials...


Batches:   0%|          | 0/10 [00:00<?, ?it/s]

Success! Embeddings shape: (299, 384) (rows × 384 dimensions)

Sample cosine similarity matrix (first 5 items):
[[1.    0.986 0.983 0.981 0.886]
 [0.986 1.    0.995 0.985 0.893]
 [0.983 0.995 1.    0.982 0.888]
 [0.981 0.985 0.982 1.    0.884]
 [0.886 0.893 0.888 0.884 1.   ]]

Most similar to 'Clear Glass - 3mm thick':
  → Clear Glass - 4mm thick (score: 0.986)
  → Clear Glass - 5mm thick (score: 0.983)
  → Clear Glass - 6mm thick (score: 0.981)

Saved enriched data:
  CSV: ..\data\interior_materials_enriched_20260219_1714.csv
  Parquet: ..\data\interior_materials_enriched_20260219_1714.parquet (recommended for the next Phase)
